In [1]:
%matplotlib tk
%load_ext autoreload
%autoreload 2

#
from utils import GerbilPCA
import numpy as np
import matplotlib.pyplot as plt

/home/cat/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#root_dir = '/mnt/b3a68699-495d-4ebb-9ab1-ac74f11c68c5/gerbil/aggregate_data/fig2'
root_dir = '/home/cat/code/gerbil/meta_data/fig2/'

behaviors = [

   # #  ##NEST BEHAVIORS
   #  'exits',
   #   'huddling',

   # #  # # # ###APPROACHES
   # ### 'adult_pup_approaches',
   #  'pup_adult_approaches',
   #  'pup_pup_approaches',

   #  # ##PAIRWISE PROXMITY
   #   'pairwise_adult_pup',
   #   'pairwise_pup_pup',

    # AUTONOMY 
   'distance',
    'exploration',
   'food',
   'water'
]

In [12]:
g = GerbilPCA(root_dir, 
              behaviors)

#
g.smooth = False
g.smooth_window = 5
g.smooth_order = 2
g.standardize = False
g.n_dimensions = 2
g.scatter_size = 600

#
g.load_data()

#
g.n_samples_kde = 1000
g.sliding_window_size = 2
g.dist_method = '2sample_ks_test'  # options: t_test, 2sample_ks_test
g.pval_thresh = 0.05
g.show_plots = True
g.smooth_pval = True
g.size=500
g.interpolate_first_value = False
g.get_rapid_dev_plots()

#data
g.plot_mean_behavior()

#
g.get_area_under_curve_Fig2()


behavior:  distance (12, 14)
behavior:  exploration (12, 14)
behavior:  food (12, 14)
behavior:  water (12, 14)


100%|██████████| 4/4 [00:00<00:00, 1899.38it/s]

mean:  0.5129425364090142  std:  0.2818123533918446
mean:  0.66631102176174  std:  0.32671675125258526
mean:  0.9716016829849133  std:  0.8367028113374393
mean:  0.3489278244255058  std:  0.35387316149735215
(12, 14)
(12, 14)
(12, 14)
(12, 14)
auc_f : (4, 12)
auc_f mean:  [ 6.72110509  8.7367093  12.51487943  4.5104799 ]  ,std:  [0.67674495 1.13146915 3.35604301 1.12542997]
anova:  F_onewayResult(statistic=35.65030451813826, pvalue=7.612729167613185e-12)


In [28]:
#
g.show_convex_hull = True
g.remove_outlier_point= False
g.show_cohorts = True


#
g.run_pca()


self.stack:  10


In [29]:
#
g.find_overlaps()


In [36]:
# compute KL distance across deveopmental windows
# split the data at 2 differnt time points and then compute the KL for sequential chunks


#
g.compute_dev_stages_histogram()

#
#g.plot_summary_devs()


In [46]:
#
#g.compute_mahalanobis_distance_distributions()
#
# g.n_bins = 13
# g.compute_mahalanobis_distance_histograms()



In [7]:
g.show_dev_stage_similarties()

(4, 14)


In [68]:

def replace_isolated_ones_with_zeros(temp):
    
    # This switches the first value if its followed by a 2+ sequence
    start_idx = 1
    if temp[start_idx]!=temp[start_idx+1]:
        print(temp[start_idx], temp[start_idx+1], temp[start_idx+2])
        if temp[start_idx+1]==temp[start_idx+2]:
            temp[start_idx] = temp[start_idx+1]

    # rest of the values
    for k in range(start_idx+1,temp.shape[0]-1):
        #
        if temp[k]==temp[k-1]:
            continue
        elif temp[k]!=temp[k+1]:
            temp[k] = temp[k+1]

    # 
    return temp


temp = np.zeros(14)

#temp [2:4] = 1
temp[1] = 1
temp[4] = 1

temp[6] = 1
temp[7] = 1

print (temp)

temp = replace_isolated_ones_with_zeros(temp)
print (temp)


[0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0.]
1.0 0.0 0.0
[0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0.]


In [85]:
#
from tqdm import trange
import parmap

# compute KL distance across deveopmental windows
# split the data at 2 differnt time points and then compute the KL for sequential chunks
g.behavior_id = 3
g.n_samples_kde = 1000
#g.dist_method = 'js_div'
#g.dist_method = 'gausianity'
g.dist_method = 'wasserstein_distance'
#g.dist_method = 'kl_div'


#
n_runs = 30
#
if True:
    res = parmap.map(g.get_kl_matrix, 
                     range(n_runs), 
                     pm_processes=10,
                     pm_pbar=True)
    
    t1_m = []
    t2_m = []
    for k in range(len(res)):
        t1_m.append(res[k][0])
        t2_m.append(res[k][1])

else:
    t1_m = []
    t2_m = []
    for k in trange(n_runs):
        t1,t2 = g.get_kl_matrix(k)
        t1_m.append(t1)
        t2_m.append(t2)

#
t1_m = np.array(t1_m)
t2_m = np.array(t2_m)

# take averages over axis 0
t1_m = np.nanmean(t1_m,0)
t2_m = np.nanmean(t2_m,0)

#
labels = np.arange(16,30,1)
plt.figure()
plt.subplot(121)
plt.imshow(t1_m,
           # use divergent colormap with white in the middle
            #  cmap='RdBu_r',
           )
plt.xticks(np.arange(14),labels)
plt.yticks(np.arange(14),labels)
plt.colorbar()
plt.xlabel("t2")
plt.ylabel("t1")

#
plt.subplot(122)
plt.imshow(t2_m,
           #cmap='RdBu_r',
           )
plt.xticks(np.arange(14),labels)
plt.yticks(np.arange(14),labels)
plt.colorbar()

# rewrite xlabel so it starts at 15 instead of 0    
# 
plt.xlabel("t2")
plt.ylabel("t1")

#
plt.show()
        

100%|██████████| 30/30 [00:04<00:00,  7.15it/s]
/tmp/ipykernel_140888/892830536.py:43: RuntimeWarning: Mean of empty slice
  t1_m = np.nanmean(t1_m,0)
/tmp/ipykernel_140888/892830536.py:44: RuntimeWarning: Mean of empty slice
  t2_m = np.nanmean(t2_m,0)


In [186]:
# plot the distribution of points from day 0 to 8
behavvior_id = 0
temp1 = g.data[behavior_id][:,:8]
temp2 = g.data[behavior_id][:,8:8+3]
print (temp1.shape, temp2.shape)

plt.figure()
plt.plot(np.mean(temp1,0))
plt.plot(np.mean(temp2,0))
plt.show()

plt.figure()
plt.scatter(temp1*0,
            temp1,
            c='blue',
            #alpha=.1,
            s=1)
plt.scatter(temp2*0+1,
            temp2,
            c='red',
            #alpha=.1,
            s=1)

plt.show()



(12, 8) (12, 3)


In [94]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Sample data (replace with your data)
#data = np.random.uniform(0, 10, 1000)

# Perform kernel density estimation (KDE) with an Epanechnikov kernel
data = temp1.copy()
kde = sm.nonparametric.KDEUnivariate(data)
kde.fit()  # Use "epa" for Epanechnikov kernel

# Evaluate the estimated density at a set of values

x = np.linspace(min(data), max(data), 1000)
pdf = kde.evaluate(x)

# get weighted samples from the KDE
# Generate random samples from the estimated density
num_samples = 1000
samples = np.random.choice(x, size=num_samples, p=pdf/pdf.sum())


# Plot the estimated density
plt.plot(x, pdf)
plt.hist(samples, density=True, alpha=0.5)
plt.title("Kernel Density Estimation (Epanechnikov Kernel)")
plt.xlabel("x")
plt.ylabel("Density")
plt.show()


In [40]:
# Step 2: Compute the convex hull
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import numpy as np

def plot_polyhedron(ax,temp, ctr):

    clrs = ['black','blue','red']
    hull = ConvexHull(temp)
    vertices = temp[hull.vertices]  # Vertices of the convex hull

    # get the ids of the vertices
    # make a larger vertices array and fill in the missing vertices with zeros
    vertices = np.zeros((temp.shape[0],3))
    vertices[hull.vertices] = temp[hull.vertices]
    #

    # Step 3: Extract the valid edges and vertices from the convex hull
    edges = []
    for simplex in hull.simplices:
        n_bad = 0
        for v in simplex:
            #print (v)
            if v < 0 or v >= len(temp):
                n_bad += 1

        if n_bad == 0:
            edges.append(simplex)

    # Step 5: Create a Poly3DCollection for the 3D polygon
    poly3d = [[vertices[edge[0]], vertices[edge[1]], vertices[edge[2]]] for edge in edges]

    ax.add_collection3d(Poly3DCollection(poly3d, 
                                        facecolors=clrs[ctr], 
                                        linewidths=1, 
                                        edgecolors=clrs[ctr], 
                                        alpha=0.5))
    
#
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

plot_polyhedron(ax,g.polygon_array[0], 0)
plot_polyhedron(ax,g.polygon_array[1], 1)
plot_polyhedron(ax,cc.points(), 2)

ax.set_xlim(-10,10)
ax.set_ylim(-10,10)
ax.set_zlim(-10,10)


plt.show()

In [32]:
print (cc.points())

[[-0.702962    0.7062842  -0.26558292]
 [-0.8063882  -0.17914958 -0.3446897 ]
 [-0.81119835 -0.28091145 -0.35484204]
 [-0.7396896   1.3807548  -0.36920157]
 [-0.85529685 -0.37352386 -0.4178022 ]
 [-1.0109018  -0.8226008  -0.6443445 ]
 [-1.1583123  -0.07611743 -0.85183734]
 [-1.0330684   0.45338902 -0.71459347]
 [-1.0984539  -0.5606807  -0.7533681 ]
 [-1.1302632  -0.37038487 -0.78956896]]


In [249]:
g.plot_overlap_volumes()


polygon array:  [array([[ 1.10918594,  0.95192086,  2.28657032],
       [ 4.38407873, -0.23680002,  1.25295694],
       [ 3.90056015,  0.63616137,  1.36920365],
       [ 0.05088653,  6.13126071,  0.38957656],
       [ 5.4099821 , -3.95657028, -0.85071022],
       [ 2.85869699,  0.65861864, -0.42668024],
       [-1.46358674, -0.56719916, -1.24742684],
       [-1.74267848, -1.59532024, -1.66173092],
       [ 1.16584881, -0.14581732, -2.68784599]]), array([[-0.53556641, -0.44382364, -0.41356438],
       [-1.13124266, -0.09174938, -0.28665785],
       [-0.31425656, -0.36563015, -1.03345376],
       [-0.88934806,  1.98686823, -0.10081344],
       [ 0.21566913, -1.4671115 , -0.92678335],
       [-0.51894257,  0.48673448, -0.76507539],
       [-3.27458569,  0.36686849, -0.12309288],
       [-2.43665007,  0.36235476, -0.57677616]])]


AttributeError: module 'Geometry3D' has no attribute 'ConvexHull'

[[ 1.10918594  0.95192086  2.28657032]
 [ 4.38407873 -0.23680002  1.25295694]
 [ 3.90056015  0.63616137  1.36920365]
 [ 0.05088653  6.13126071  0.38957656]
 [ 5.4099821  -3.95657028 -0.85071022]
 [ 2.85869699  0.65861864 -0.42668024]
 [-1.46358674 -0.56719916 -1.24742684]
 [-1.74267848 -1.59532024 -1.66173092]
 [ 1.16584881 -0.14581732 -2.68784599]]
0.0


In [57]:
#
g.run_umap()

(198, 2)


In [231]:
import numpy as np
import triintersect

# Define the vertices of the two polygons
polygon1 = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])
polygon2 = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])

# Triangulate the polygons
triangles1 = triintersect.triangulate(polygon1)
triangles2 = triintersect.triangulate(polygon2)

# Initialize a variable to store the intersection volume
intersection_volume = 0

# Iterate through the triangles of the first polygon
for triangle1 in triangles1:
    # Iterate through the triangles of the second polygon
    for triangle2 in triangles2:
        # Compute the intersection of the two triangles
        intersection = triintersect.intersection(triangle1, triangle2)
        if intersection:
            # Calculate the volume of the intersection and add it to the total
            intersection_volume += triintersect.volume(intersection)

print("Intersection Volume:", intersection_volume)


ModuleNotFoundError: No module named 'triintersect'

In [26]:
g.n_dimensions = 2
g.fit_gmm_pca()



(48, 14)
(48, 2)


In [27]:
#


(48, 2)


In [2]:
import numpy as np
d = np.load('/home/cat/Downloads/Downloads/pup_pup_pairwise.npy')
print (d.shape)

(18, 14)
